## Text Classification using TensorFlow

In this task, I'll guide you through the process of text classification using transfer learning with TensorFlow Hub and Keras. We'll be working with the IMDB dataset, which comprises 50,000 movie reviews sourced from the Internet Movie Database. These reviews are split into 25,000 for training and 25,000 for testing. Importantly, both the training and test sets are balanced, containing an equal number of positive and negative reviews.

Let's begin this text classification task with TensorFlow by importing some essential libraries:

In [1]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 20.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 16.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21484 sha256=9fb26595388352b682ebcaae316176eed3f69568dc955708e2871b153e232b84
  Stored in directory: /Users/m.daoudadala/Library/Caches/pip/wheels/54/4e/28/3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling 

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


You can load the IMDB dataset directly from TensorFlow and split it into training and test sets as follows:

In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 100%|██████████| 1/1 [00:21<00:00, 21.98s/ url]


Dataset imdb_reviews downloaded and prepared to /Users/m.daoudadala/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### Data Exploration

To gain an understanding of the dataset, let's inspect the first 10 samples by printing them:

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Let's print the first 10 labels from the dataset:

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

### Constuct the model

To construct a model for Text Classification with TensorFlow, we'll utilize a pre-trained model offered by TensorFlow known as TensorFlow Hub. Let's begin by creating a Keras layer that employs a TensorFlow Hub model to embed sentences, and then test it with some sample input:

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

Now, let's build the model using the complete dataset:

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compile the model

Now, let's compile the model using the specified loss function and the Adam optimizer:

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Training of the Text Classification Model

Train the model for 20 epochs using mini-batches of 512 samples. This involves 20 iterations over all the samples of the tensors x_train and y_train. Monitor the model's loss and accuracy on the 10,000 samples in the validation set during training.

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 2s 44ms/step - loss: 1.7209 - accuracy: 0.5007 - val_loss: 0.8010 - val_accuracy: 0.5003
Epoch 2/20
30/30 [==============================] - 1s 33ms/step - loss: 0.6959 - accuracy: 0.5576 - val_loss: 0.6656 - val_accuracy: 0.5849
Epoch 3/20
30/30 [==============================] - 1s 32ms/step - loss: 0.6426 - accuracy: 0.5791 - val_loss: 0.6315 - val_accuracy: 0.5897
Epoch 4/20
30/30 [==============================] - 1s 32ms/step - loss: 0.6125 - accuracy: 0.6264 - val_loss: 0.6053 - val_accuracy: 0.6384
Epoch 5/20
30/30 [==============================] - 1s 32ms/step - loss: 0.5857 - accuracy: 0.6578 - val_loss: 0.5821 - val_accuracy: 0.6768
Epoch 6/20
30/30 [==============================] - 1s 37ms/step - loss: 0.5607 - accuracy: 0.6881 - val_loss: 0.5586 - val_accuracy: 0.6979
Epoch 7/20
30/30 [==============================] - 1s 36ms/step - loss: 0.5350 - accuracy: 0.7131 - val_loss: 0.5346 - val_accuracy: 0.7103
Epoch 8/20
30

### Evaluate The Model

After training the model, let's evaluate its performance. Two values will be returned: the loss and the accuracy rate.

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.3317 - accuracy: 0.8496 - 1s/epoch - 23ms/step
loss: 0.332
accuracy: 0.850


Our Text Classification Model achieved an accuracy rate of 85 percent, which is generally appreciated.